
AB Test consumer APP walkaround

In [5]:
import pandas as pd

from config import (
    BASELINE_REDEMPTION_RATE,
    TARGET_REDEMPTION_RATE,
    REQUIRED_SAMPLE_PER_VARIANT,
)

from analysis.metrics import compute_variant_metrics
from analysis.test import run_redemption_ztest


Loading Data (Dummy)

In [7]:

data = {
    "user_id": range(1, 100001),
    "variant": ["control"] * 50000 + ["treatment"] * 50000,
    "redeemed": (
        ([0, 1] * 25000) +                    # Control: exactly 50,000
        ([0, 1, 1] * 16666 + [0, 0])          # Treatment: 49,998 + 2 = 50,000
    )
}

df = pd.DataFrame(data)


Metrics

In [10]:
summary = compute_variant_metrics(df)
summary


,variant,users,redeemers,redemption_rate
0,control,50000,25000,0.50000
1,treatment,50000,33332,0.66664


Sample Size Check

In [11]:
summary[["variant", "users"]]


,variant,users
0,control,50000
1,treatment,50000


Test

In [12]:
test_results = run_redemption_ztest(summary)
test_results


{'z_stat': np.float64(-53.44345172992486), 'p_value': np.float64(0.0)}

In [13]:
summary["uplift_vs_control"] = (
    summary["redemption_rate"] /
    summary.loc[summary["variant"] == "control", "redemption_rate"].values[0]
    - 1
)

summary


,variant,users,redeemers,redemption_rate,uplift_vs_control
0,control,50000,25000,0.50000,0.00000
1,treatment,50000,33332,0.66664,0.33328
